In [ ]:
import os
import json
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk

In [ ]:
base_dir = "/"

In [2]:
# 데이터셋 로드
dataset_dict = load_from_disk(os.path.join(base_dir, "data", "test_dataset"))

# 'test' 데이터셋을 선택하고 Pandas 데이터프레임으로 변환
test_dataset = dataset_dict['validation']
df = test_dataset.to_pandas()

In [ ]:
model_id = "rtzr/ko-gemma-2-9b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model.eval()

In [5]:
instruction = """다음의 '제시 답변'에서 필요한 답만 동일하게 추출하여 제시하시오. 추가적인 정보나 수정 없이, '제시 답변'의 내용을 사용하시오.


다음을 반드시 지키시오: 필요시 조사, 어미를 제거할 수 있으나, '제시 답변' 이외의 내용을 생성, 추가하면 절대 안됨. 중요 핵심 내용을 남기고 제거할 수 있다.

### 예시 ###
질문: 제2차 세계 대전은 몇 년에 발발하였는가?  제시 답변: 1939년 9월 1일에 발발하였다.
생성 답변: 1939년

### 질문 ###
{}

### 제시 답변 ###
{}"""

In [6]:
with open("predictions.json", "r") as f:
    predictions = json.load(f)

In [ ]:
new_pred = {}

for i in range(len(df)):
    question = df.loc[i, 'question']

    messages = [
        {"role": "user", "content": instruction.format(question, predictions[df.loc[i, "id"]])},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<end_of_turn>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=2048,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    pred = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    new_pred[df.loc[i, "id"]] = pred.strip()

In [13]:
with open(os.path.join(base_dir, "data", "predictions_fixed.json"), "w") as f:
    json.dump(new_pred, f, ensure_ascii=False, indent=4)